In [13]:
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

Defaulting to user installation because normal site-packages is not writeable


In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit =5
generator = square_root_generator(limit)
answer = 0
for sqrt_value in generator:
    answer+=sqrt_value
    print(answer)
    #print(sqrt_value)

1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762


In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit =13
generator = square_root_generator(limit)
answer = 0
for sqrt_value in generator:
    answer+=sqrt_value
    #print(answer)
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


In [37]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

In [38]:




def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)




def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)




{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [51]:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace" 
                                        )

print(info)


info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/Ethan/Data_Ingestion_week3/Workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707262799.2518358 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/Ethan/Data_Ingestion_week3/Workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707262799.8189814 is LOADED and contains no failed jobs


In [52]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")


conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


rides = conn.sql("SELECT sum(age) FROM people").df()
display(rides)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,sum(age)
0,353.0


In [57]:
info = generators_pipeline.run(people_1(),
										table_name="people2",
										write_disposition="replace" ,
                                        primary_key="ID"
                                        )

print(info)


info = generators_pipeline.run(people_2(),
										table_name="people2",
										write_disposition="merge", 
                                        primary_key="ID")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/Ethan/Data_Ingestion_week3/Workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707263528.4171999 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////home/Ethan/Data_Ingestion_week3/Workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707263529.0278184 is LOADED and contains no failed jobs


In [58]:


conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


rides = conn.sql("SELECT sum(age) FROM people2").df()
display(rides)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people2             │
└─────────────────────┘

,sum(age)
0,266.0
